In [2]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

ANALISI CULTURA E SPETTACOLO SENZA PESI

Utilizzeremo i seguenti dati: 

1. **Musei**
2. **Cinema**
3. **Teatri**

Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

In [4]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_MUSEI_CLEAN),
     "tipo": "Musei",
     "gdf":  gpd.read_file(PATH_CINEMA_CLEAN),
     "tipo": "Cinema", 
     "gdf":  gpd.read_file(PATH_TEATRI_CLEAN),
     "tipo": "Teatri"}
     ]
Graph_functions.auto_analysis_poi(gdf_list, custom_weights={"default": 1}, PATH_GEOJSON="../Data/Clean/Analisi/Cultura_e_spettacolo_nopesi.geojson")

In [6]:
gdf_analisi_cultura_spettacolo_nopesi = gpd.read_file("../Data/Clean/Analisi/Cultura_e_spettacolo_nopesi.geojson", driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [8]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_cultura_spettacolo_nopesi_mod = gdf_analisi_cultura_spettacolo_nopesi[gdf_analisi_cultura_spettacolo_nopesi["artificial"] != True]

# Leggi i due POI
gdf_musei = gpd.read_file(PATH_MUSEI_CLEAN)
gdf_teatri = gpd.read_file(PATH_TEATRI_CLEAN)
gdf_cinema = gpd.read_file(PATH_CINEMA_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_cinema.geometry, gdf_teatri.geometry, gdf_musei.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_cultura_spettacolo_nopesi.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_cultura_spettacolo_nopesi_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Salviamo per test di visualizzazione su kepler
gdf_analisi_cultura_spettacolo_nopesi_mod.to_file("../Data/Clean/Analisi/Cultura_e_spettacolo_nopesi_senza_poi.geojson", driver="GeoJSON")

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_cultura_spettacolo_nopesi_mod_most_highways = gdf_analisi_cultura_spettacolo_nopesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [9]:
# Stampiamo il risultato
print(gdf_analisi_cultura_spettacolo_nopesi_mod_most_highways)
print(f"Tot: {gdf_analisi_cultura_spettacolo_nopesi_mod_most_highways["length"].sum()}")

                             length
highway                            
footway                   21.007829
residential               12.861390
unclassified               8.668459
tertiary                   7.012068
secondary                  4.497550
pedestrian                 1.675621
service                    1.149832
primary                    0.785486
cycleway                   0.577529
footway steps              0.203335
tertiary_link              0.180138
residential pedestrian     0.158752
secondary_link             0.127444
path                       0.124023
residential service        0.085555
steps                      0.079204
footway service            0.078886
unclassified primary       0.067041
unclassified residential   0.066621
unclassified tertiary      0.048681
primary_link               0.026263
Tot: 59.48170546839288
